In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
import numpy
model = Sequential()
model.add(Dense(150, input_dim=122, activation='tanh'))
model.add(Dropout(0.1))
model.add(Dense(120, activation='relu'))
model.add(Dropout(0.01))
model.add(Dense(50, activation='sigmoid'))

model.add(Dense(40, activation='softmax'))
# Compile model
from keras import optimizers
# fix random seed for reproducibility
numpy.random.seed(7)

Using TensorFlow backend.


In [41]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('KDD_Dataset.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 41:42].values

In [42]:
from sklearn.preprocessing import LabelEncoder
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

In [43]:
y

array([16, 16, 14, ...,  1, 16, 11])

In [44]:
import category_encoders as ec

In [45]:
ds = dataset.iloc[:,:-1].select_dtypes(include=['object']).copy()

In [46]:
ds3 = dataset.iloc[:,:-1].select_dtypes(exclude=['object']).copy()

In [47]:
dataset_ec = ec.OneHotEncoder(cols = ['duration', 'protocol_type','service'])
ds2 = dataset_ec.fit_transform(ds, X)

In [48]:
ds2.head()

,duration_1,duration_2,duration_3,protocol_type_1,protocol_type_2,protocol_type_3,protocol_type_4,protocol_type_5,protocol_type_6,protocol_type_7,...,service_2,service_3,service_4,service_5,service_6,service_7,service_8,service_9,service_10,service_11
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
X = pd.concat([ds3, ds2], axis = 1)

In [50]:
X.head()

,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,service_2,service_3,service_4,service_5,service_6,service_7,service_8,service_9,service_10,service_11
0,491,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,146,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
0,232,8153,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
0,199,420,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2, 
                                                    random_state = 1)


In [53]:
len(set(y))

40

In [54]:
# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 500, 
                                    criterion = 'entropy', 
                                    random_state = 0,
                                    oob_score = True)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# K-fold CV
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator= classifier, 
                             X = X_train,
                             y = y_train,
                             cv=10)

accuracies_mean = accuracies.mean()
accuracies_std = accuracies.std()

print("Accuracy and STD of RF")
print(accuracies_mean)
print(accuracies_std)

Accuracy and STD of RF
0.9969789075941755
0.0007012062143864484


In [55]:
#Keras code starts here
import datetime, os
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
sgd = optimizers.SGD(lr=0.0001, decay=0.0, momentum=0.9, nesterov=True)

# pass optimizer by name: default parameters will be used
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# All parameter gradients will be clipped to
# a maximum norm of 1.
#sgd = optimizers.SGD(lr=0.0001, clipnorm=1.)
# Fit the model
history = model.fit(X_train, y_train,
          batch_size=5,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=None)


Train on 118813 samples, validate on 29704 samples
Epoch 1/10
118813/118813 [==============================] - 44s 374us/step - loss: 0.2850 - accuracy: 0.9187 - val_loss: 0.2144 - val_accuracy: 0.9368
Epoch 2/10
118813/118813 [==============================] - 44s 369us/step - loss: 0.2107 - accuracy: 0.9366 - val_loss: 0.2123 - val_accuracy: 0.9444
Epoch 3/10
118813/118813 [==============================] - 44s 374us/step - loss: 0.1957 - accuracy: 0.9408 - val_loss: 0.2051 - val_accuracy: 0.9342
Epoch 4/10
118813/118813 [==============================] - 45s 380us/step - loss: 0.1897 - accuracy: 0.9426 - val_loss: 0.1956 - val_accuracy: 0.9421
Epoch 5/10
118813/118813 [==============================] - 40s 336us/step - loss: 0.1860 - accuracy: 0.9441 - val_loss: 0.1630 - val_accuracy: 0.9531
Epoch 6/10
118813/118813 [==============================] - 39s 329us/step - loss: 0.1766 - accuracy: 0.9477 - val_loss: 0.1635 - val_accuracy: 0.9599
Epoch 7/10
118813/118813 [=================

In [ ]:
import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()